# Covid-19 Model Training
Train the model with covid-19 datasets <br>
<br>
**Important note**: please run this code in Google Colab, but if you want to use other don't forget to adjust to their own version code

In [1]:
!pip install translate

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imageio
import pathlib
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
from tensorflow.keras import callbacks, optimizers
from google.colab import drive

%matplotlib inline

In [3]:
drive.mount("/content/drive", force_remount=True)

MessageError: ignored

In [ ]:
!ls

In [ ]:
%cd drive/MyDrive/CovidDatasets

In [ ]:
!unzip archive.zip

In [ ]:
from condition import condition

In [ ]:
condition

In [ ]:
%cd raw_image/

In [ ]:
os.listdir('train')

In [ ]:
!ls train

In [ ]:
for i in os.listdir('train/'):
  print(i, len(os.listdir('train/' + i)))

In [ ]:
def img_data(dir_path, target_size, batch, class_lst, preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()
  
  return (gen_object.flow_from_directory(dir_path, 
                                        target_size=target_size, 
                                        batch_size=batch, 
                                        class_mode='sparse', 
                                        classes=class_lst,
                                        shuffle = True))

In [ ]:
train_data_gen = img_data("train/", (224, 224), 500, os.listdir('train'), preprocess_input)
valida_data_gen = img_data("test/", (224, 224), 500, os.listdir('test'), preprocess_input)

In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000,
    classifier_activation='softmax'
)

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_check = callbacks.ModelCheckpoint('.model.hdf5', save_best_only=True, monitor='val_loss', mode=min)


In [ ]:
model.fit(train_data_gen, batch_size=500, validation_data=valida_data_gen, callbacks=[elst, save_check], epochs=30)